In [5]:
%cd lit-gpt

/home/aniket/workspace/aniket-codes/2023-10-11 LLMs-AICamp-London/lit-gpt


In [ ]:
# !wget https://gist.githubusercontent.com/aniketmaurya/b8e5bd3f1594bd31ed34375ed916f075/raw/b831e5ab054ac6c94a4a409d32be14b6fdcad82a/databricks-dolly-15k.csv
# !mv databricks-dolly-15k.csv /data/aniket/datasets/databricks-dolly-15k.csv

In [1]:
import pandas as pd

df = pd.read_csv("/data/aniket/datasets/databricks-dolly-15k.csv")

In [2]:
df.head()

,instruction,input,output
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...
1,Which is a species of fish? Tope or Rope,NaN,Tope
2,Why can camels survive for long without water?,NaN,Camels use the fat in their humps to keep them...
3,"Alice's parents have three daughters: Amy, Jes...",NaN,The name of the third daughter is Alice
4,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981."


In [4]:
COLUMNS = ["instruction", "input", "output"]
df[COLUMNS].to_csv("databricks-dolly-15k.csv", index=False)

In [6]:
!python scripts/prepare_csv.py \
    --csv_path "databricks-dolly-15k.csv" \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --destination_path "data/dolly" \
    --max_seq_length 512

Loading tokenizer...
train has 13,510 samples
test has 1,501 samples
Processing train split ...
100%|███████████████████████████████████| 13510/13510 [00:12<00:00, 1113.75it/s]
Processing test split ...
100%|█████████████████████████████████████| 1501/1501 [00:01<00:00, 1141.28it/s]


In [7]:
!python finetune/lora.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --data_dir "data/dolly" \
    --out_dir "out/lora/dolly"

Using bfloat16 Automatic Mixed Precision (AMP)
{'eval_interval': 100, 'save_interval': 100, 'eval_iters': 100, 'eval_max_new_tokens': 100, 'log_interval': 1, 'devices': 1, 'learning_rate': 0.0003, 'batch_size': 128, 'micro_batch_size': 1, 'gradient_accumulation_iters': 128, 'max_iters': 50000, 'weight_decay': 0.01, 'lora_r': 8, 'lora_alpha': 16, 'lora_dropout': 0.05, 'lora_query': True, 'lora_key': False, 'lora_value': True, 'lora_projection': False, 'lora_mlp': False, 'lora_head': False, 'warmup_steps': 100}
Seed set to 1337
Loading model '/data/aniket/Llama-2-7b-hf/lit_model.pth' with {'org': 'meta-llama', 'name': 'Llama-2-7b-hf', 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 64, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 32, 'shared_attention_norm': False, '_norm_class': 'RMSNorm', 'norm_eps': 1e-05, '_mlp_class': 'LLaMAMLP', 'gel

In [8]:
!python finetune/lora.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --data_dir "data/dolly" \
    --out_dir "out/lora/dolly" \
    --precision bf16-true

{'eval_interval': 100, 'save_interval': 100, 'eval_iters': 100, 'eval_max_new_tokens': 100, 'log_interval': 1, 'devices': 1, 'learning_rate': 0.0003, 'batch_size': 128, 'micro_batch_size': 1, 'gradient_accumulation_iters': 128, 'max_iters': 50000, 'weight_decay': 0.01, 'lora_r': 8, 'lora_alpha': 16, 'lora_dropout': 0.05, 'lora_query': True, 'lora_key': False, 'lora_value': True, 'lora_projection': False, 'lora_mlp': False, 'lora_head': False, 'warmup_steps': 100}
Seed set to 1337
Loading model '/data/aniket/Llama-2-7b-hf/lit_model.pth' with {'org': 'meta-llama', 'name': 'Llama-2-7b-hf', 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 64, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 32, 'shared_attention_norm': False, '_norm_class': 'RMSNorm', 'norm_eps': 1e-05, '_mlp_class': 'LLaMAMLP', 'gelu_approximate': 'none', 'intermediate_size': 11

In [ ]:
!python finetune/lora.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --data_dir "data/dolly" \
    --out_dir "out/lora/dolly" \
    --precision bf16-true \
    --quantize bnb.fp4 

In [ ]:
!python scripts/merge_lora.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --lora_path "out/dolly/Llama-2-7b-hf/lit_model_lora_finetuned.pth" \
    --out_dir "out/dolly/Llama-2-7b-hf/"


!python eval/lm_eval_harness.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --eval_tasks "[truthfulqa_mc]" \
    --precision "bf16-true" \
    --batch_size 4 \
    --save_filepath "results.json"